In [1]:
# pytorch
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.utils.data import random_split
from torch.optim import lr_scheduler
# # 自定义模块
# from torchLearning import *
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn import cluster
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
import time
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.tree import DecisionTreeClassifier
import inspect, re
from sklearn.preprocessing import PolynomialFeatures
from itertools import product
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm
import gc
from sklearn.feature_selection import VarianceThreshold,r_regression,SelectKBest,f_regression,SelectPercentile,chi2,f_classif,mutual_info_regression,mutual_info_classif,RFE,RFECV,SequentialFeatureSelector,SelectFromModel
from scipy import stats
import scipy
import random
from scipy.special import digamma
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,RepeatedKFold,KFold
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import roc_curve
import math

# 绘图模块
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

# 导入以下包从而使得可以在jupyter中的一个cell输出多个结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# X_train = pd.read_csv("data/X_train.csv")
# X_test = pd.read_csv("data/X_test.csv")
# y_train = pd.read_csv("data/y_train.csv")
# y_test = pd.read_csv("data/y_test.csv")
data=pd.read_csv('newdata.csv')
data.fillna(data.mean(), inplace=True)
# 假设 data 是已经加载的DataFrame
# data = pd.read_csv('your_dataset.csv')  # 示例代码，假设您已经加载了数据

# 离散字段
category_cols = [
    "gender", "start_time_DI", "last_event_DI","final_cc_cname_DI","LoE_DI","course_id", "userid_DI"
]

# 连续字段
numeric_cols = [
    "registered", "viewed",  "explored", "certified", "YoB",
    "nevents", "ndays_act", "nplay_video","nchapters","nforum_posts"
]
# 标签
Target = "grade"

train, test = train_test_split(data, random_state=55914)
X_train = train.drop(columns=["grade"]).copy()
X_test = test.drop(columns=["grade"]).copy()

y_train = train['grade'].copy()
y_test = test['grade'].copy()

ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ord_enc.fit(X_train[category_cols])
# ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index

X_train = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_32668\4222964018.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)


OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

In [3]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, sequence_length):
        super(BiLSTM, self).__init__()
        self.sequence_length = sequence_length
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.bilstm1 = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)  # 双向LSTM
        self.dropout1 = nn.Dropout(0.3)
        self.bilstm2 = nn.LSTM(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True)  # 双向LSTM
        self.dropout2 = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size * 2, output_size) 
        self.relu = nn.ReLU()  # 添加ReLU激活函数
    def forward(self, x):
        x, _ = self.bilstm1(x)
        x = self.relu(x)
#         x = self.dropout1(x)
        x, _ = self.bilstm2(x)
        x = self.relu(x)
#         x = self.dropout2(x)
        x = self.fc(x[:, -1, :])  # Get the output from the last time step
        x = torch.squeeze(x, 1)  # Squeeze the output to ensure the dimension is (batch_size, output_size)
        return x

In [4]:
X_train.shape

(8791, 17)

In [5]:
y_train = y_train.squeeze()
y_test = y_test.squeeze()
y_train.shape

(8791,)

In [6]:
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)


In [7]:
# 检查 X_scaled 中是否有 NaN 值
has_nan = np.isnan(X_train).any()
nan_columns = np.isnan(X_train).any(axis=0)

# 输出结果
has_nan, np.where(nan_columns)[0]

(gender               False
 start_time_DI        False
 last_event_DI        False
 final_cc_cname_DI    False
 LoE_DI               False
 course_id            False
 userid_DI            False
 registered           False
 viewed               False
 explored             False
 certified            False
 YoB                  False
 nevents              False
 ndays_act            False
 nplay_video          False
 nchapters            False
 nforum_posts         False
 dtype: bool,
 array([], dtype=int64))

In [8]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import confusion_matrix, classification_report

# 特征缩放
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [9]:
# # 检查每个特征的方差
# variances = X_scaled.var(axis=0)
# zero_variance_features = np.where(variances == 0)[0]

# if zero_variance_features.size > 0:
#     print("特征方差为零的索引：", zero_variance_features)
#     # 处理这些特征，例如删除
#     X_train = np.delete(X_scaled, zero_variance_features, axis=1)
#     X_test = np.delete(X_scaled, zero_variance_features, axis=1)

In [10]:

# 检查 X_scaled 中是否有 NaN 值
has_nan = np.isnan(X_scaled).any()
nan_columns = np.isnan(X_scaled).any(axis=0)

# 输出结果
has_nan, np.where(nan_columns)[0]

(False, array([], dtype=int64))

In [13]:
# Convert to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Assuming the target is categorical
# Convert to tensors
X_tensor_test = torch.tensor(X_scaled_test, dtype=torch.float32)
y_tensor_test = torch.tensor(y_test.values, dtype=torch.long)  # Assuming the target is categorical
# Create a dataset
train_dataset = TensorDataset(X_tensor, y_tensor)
test_dataset = TensorDataset(X_tensor_test, y_tensor_test)

criterion = nn.CrossEntropyLoss()

# 数据加载
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=True)

# 定义评估指标
scoring = make_scorer(accuracy_score)

# 定义参数空间
param_grid = {
    'hidden_size': [256],
    'dropout1': [0.3],
    'dropout2': [0.5],
    'dropout3': [0.4],
    'learning_rate': [0.0005]
}
def calculate_metrics(y_true, y_pred):
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    return f1, recall, auc

Accuracy, F1, Recall, Auc = 0, 0, 0, 0
accuracy_list = []
accuracy1_list = []
accuracy_train_list = []
best_params = {}

for params in ParameterGrid(param_grid):
    model = BiLSTM(input_size=X_train.shape[1], hidden_size=params['hidden_size'], 
                           output_size=2, sequence_length=3)
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    
    # 训练模型
    num_epochs = 150
    for epoch in tqdm(range(num_epochs), desc='Epoch'):
        total_train = 0
        correct_train = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()   
            inputs = inputs.unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            # 梯度裁剪
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=20)  # 设置梯度的最大范数为20
            optimizer.step()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)  # 累加总样本数
        correct_train += (predicted == labels).sum().item()
        accuracy_train = 100 * correct_train / total_train 
        accuracy_train_list.append(accuracy_train)  # 记录准确率
        # 评估模型
        correct = 0
        total = 0
        y_true = []
        y_pred = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.unsqueeze(1)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)  # 累加总样本数
                
                correct += (predicted == labels).sum().item()
                y_true.extend(labels.tolist())
                y_pred.extend(predicted.tolist())
        Accuracy = correct / total 
        accuracy_list.append(accuracy)  # 记录准确率
        F1, Recall, Auc = calculate_metrics(y_true, y_pred)
   

Epoch: 100%|█████████████████████████████████████████████████████████████████████████| 150/150 [11:34<00:00,  4.63s/it]


In [14]:
print(f'Accuracy:{Accuracy:.4f},F1_score: {F1:.4f}, Recall: {Recall:.4f}, AUC: {Auc:.4f}')

Accuracy:0.9724,F1_score: 0.9737, Recall: 0.9715, AUC: 0.9724
